In [3]:
import pandas as pd
from imdb_dataset import IMDbDataset
import re
import numpy as np

In [2]:
imdb = IMDbDataset("all_datasets/imdb/")
movies = pd.read_csv('all_datasets/movielens/movies.csv')
links = pd.read_csv('all_datasets/movielens/links.csv')

In [22]:
# get top 3 actors dataframe
top_3_actors = imdb.cast.groupby('imdbId').head(3)
top_3_actors['actor_rank'] = top_3_actors.groupby('imdbId').cumcount() + 1
top_3_actors = top_3_actors.pivot(index='imdbId', columns='actor_rank', values='actorId').reset_index()
top_3_actors.columns = ['imdbId', 'actor1', 'actor2', 'actor3']

In [45]:
# merge with IMDb dataset to get extra movies info
extra_info = imdb.basics.merge(imdb.directors, on='imdbId', how='outer').merge(imdb.ratings, on='imdbId', how='outer').merge(top_3_actors, on='imdbId', how='outer')

In [47]:
links['imdbId'] = links['imdbId'].astype(str).str.zfill(7)
extra_info['imdbId'] = extra_info['imdbId'].str.lstrip('t')
movies['title']= movies['title'].str.strip()

C:\Users\KHAI\AppData\Local\Temp\ipykernel_3108\3338069632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extract['imdbId'] = extract['imdbId'].str.lstrip('t')


In [49]:
def extract_year(x):
    pattern = re.search('\((\d{4})\)', x)
    if pattern != None:
        return pattern.group(1)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\KHAI\AppData\Local\Temp\ipykernel_3108\2517055862.py:2: SyntaxWarning: invalid escape sequence '\('
  pattern = re.search('\((\d{4})\)', x)


In [50]:
# extract year from title
movies['extract_year'] = movies['title'].apply(lambda x: extract_year(x) if extract_year(x) != None else np.nan)

In [51]:
# final dataframe with full movies info
new_df = movies.merge(links, on='movieId').merge(extra_info, on='imdbId', how='left')
new_df['year'] = new_df['year'].fillna(new_df['extract_year'])
new_df = new_df.drop(columns='extract_year')
new_df['directorId'] = new_df['directorId'].astype('Int64')
new_df.drop(columns=['tmdbId'], inplace=True)

In [55]:
# full movies info csv file
new_df.to_csv("all_datasets/content_based/completed_movies_info.csv")